# Integrated Strategy Backtesting Framework

## Overview
This notebook integrates the **Algo Strategy Builder** patterns with the **Backtesting Validation Framework** to create a comprehensive system for:
- Testing multiple strategy patterns (Sacudida, Envolvente, Volumen Climático)
- Optimizing parameters across different assets and timeframes
- Comparing strategy performance systematically
- Validating robustness through multiple methods

## Architecture
1. **Pattern Library**: Implementation of trading patterns from Algo Strategy Builder
2. **Strategy Builder**: Flexible strategy construction system
3. **Backtesting Engine**: Comprehensive performance evaluation
4. **Optimization Framework**: Multi-strategy parameter optimization
5. **Validation Suite**: Robustness testing and validation

---

## 1. Setup and Imports

In [ ]:
import numpy as np
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
from typing import Dict, List, Tuple, Optional, Callable
import warnings
warnings.filterwarnings('ignore')

# Plot settings
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')
%matplotlib inline

print("✓ Libraries imported successfully")

---
## 2. Core Technical Indicators

In [ ]:
def ocpSma(df, periodo):
    """Calculate Simple Moving Average"""
    df = df.copy()
    df[f'sma{periodo}'] = df['Close'].rolling(window=periodo).mean()
    return df

def ocpRsi(df, periodo=14, exponencial=True):
    """Calculate RSI (Relative Strength Index)"""
    df = df.copy()
    delta = df['Close'].diff()
    gain = delta.where(delta > 0, 0)
    loss = -delta.where(delta < 0, 0)
    
    if exponencial:
        avg_gain = gain.ewm(span=periodo, adjust=False).mean()
        avg_loss = loss.ewm(span=periodo, adjust=False).mean()
    else:
        avg_gain = gain.rolling(window=periodo).mean()
        avg_loss = loss.rolling(window=periodo).mean()
    
    rs = avg_gain / avg_loss
    rsi = 100 - (100 / (1 + rs))
    df[f'rsi{periodo}'] = rsi
    return df

def ocpVolumeSma(df, periodo=20):
    """Calculate Volume Simple Moving Average"""
    df = df.copy()
    df[f'volSma{periodo}'] = df['Volume'].rolling(window=periodo).mean()
    return df

print("✓ Technical indicators defined")

---
## 3. Pattern Recognition Library

Implementation of the three main patterns from Algo Strategy Builder

In [ ]:
class PatternLibrary:
    """Library of trading patterns from Algo Strategy Builder"""
    
    @staticmethod
    def sacudida_long(df):
        """
        Sacudida Long Pattern (Shake-out)
        - Previous candle is bearish and breaks previous low
        - Current candle is bullish and closes above previous low
        """
        signals = pd.Series(False, index=df.index)
        
        for i in range(2, len(df)):
            vela2_bajista = df['Close'].iloc[i-1] < df['Open'].iloc[i-1]
            vela2_rompe_minimo = df['Low'].iloc[i-1] < df['Low'].iloc[i-2]
            vela3_alcista = df['Close'].iloc[i] > df['Open'].iloc[i]
            vela3_confirmacion = df['Close'].iloc[i] > df['Low'].iloc[i-2]
            
            if vela2_bajista and vela2_rompe_minimo and vela3_alcista and vela3_confirmacion:
                signals.iloc[i] = True
        
        return signals
    
    @staticmethod
    def sacudida_short(df):
        """
        Sacudida Short Pattern
        - Previous candle is bullish and breaks previous high
        - Current candle is bearish and closes below previous high
        """
        signals = pd.Series(False, index=df.index)
        
        for i in range(2, len(df)):
            vela2_alcista = df['Close'].iloc[i-1] > df['Open'].iloc[i-1]
            vela2_rompe_maximo = df['High'].iloc[i-1] > df['High'].iloc[i-2]
            vela3_bajista = df['Close'].iloc[i] < df['Open'].iloc[i]
            vela3_confirmacion = df['Close'].iloc[i] < df['High'].iloc[i-2]
            
            if vela2_alcista and vela2_rompe_maximo and vela3_bajista and vela3_confirmacion:
                signals.iloc[i] = True
        
        return signals
    
    @staticmethod
    def envolvente_long(df):
        """
        Bullish Engulfing Pattern
        - Previous candle is bearish
        - Current candle is bullish and engulfs previous candle
        """
        signals = pd.Series(False, index=df.index)
        
        for i in range(1, len(df)):
            vela_alcista = df['Close'].iloc[i] > df['Open'].iloc[i]
            vela_bajista_prev = df['Close'].iloc[i-1] < df['Open'].iloc[i-1]
            cierra_sobre_ap1 = df['Close'].iloc[i] >= df['Open'].iloc[i-1]
            abre_bajo_c1 = df['Open'].iloc[i] <= df['Close'].iloc[i-1]
            
            if vela_alcista and vela_bajista_prev and cierra_sobre_ap1 and abre_bajo_c1:
                signals.iloc[i] = True
        
        return signals
    
    @staticmethod
    def envolvente_short(df):
        """
        Bearish Engulfing Pattern
        - Previous candle is bullish
        - Current candle is bearish and engulfs previous candle
        """
        signals = pd.Series(False, index=df.index)
        
        for i in range(1, len(df)):
            vela_bajista = df['Close'].iloc[i] < df['Open'].iloc[i]
            vela_alcista_prev = df['Close'].iloc[i-1] > df['Open'].iloc[i-1]
            cierra_bajo_ap1 = df['Close'].iloc[i] <= df['Open'].iloc[i-1]
            abre_sobre_c1 = df['Open'].iloc[i] >= df['Close'].iloc[i-1]
            
            if vela_bajista and vela_alcista_prev and cierra_bajo_ap1 and abre_sobre_c1:
                signals.iloc[i] = True
        
        return signals
    
    @staticmethod
    def volumen_climatico_long(df, vol_multiplier=1.75, vol_period=20):
        """
        Climatic Volume Long
        - Volume > 1.75x (or custom) the 20-period SMA
        - Bullish candle
        """
        if f'volSma{vol_period}' not in df.columns:
            df = ocpVolumeSma(df, vol_period)
        
        signals = pd.Series(False, index=df.index)
        vol_climatico = df['Volume'] > df[f'volSma{vol_period}'] * vol_multiplier
        bullish_candle = df['Close'] > df['Open']
        signals = vol_climatico & bullish_candle
        
        return signals
    
    @staticmethod
    def volumen_climatico_short(df, vol_multiplier=1.75, vol_period=20):
        """
        Climatic Volume Short
        - Volume > 1.75x (or custom) the 20-period SMA
        - Bearish candle
        """
        if f'volSma{vol_period}' not in df.columns:
            df = ocpVolumeSma(df, vol_period)
        
        signals = pd.Series(False, index=df.index)
        vol_climatico = df['Volume'] > df[f'volSma{vol_period}'] * vol_multiplier
        bearish_candle = df['Close'] < df['Open']
        signals = vol_climatico & bearish_candle
        
        return signals

print("✓ Pattern library defined")

---
## 4. Strategy Builder System

Flexible system for combining patterns with filters and parameters

In [ ]:
class StrategyBuilder:
    """
    Flexible strategy builder that combines patterns with filters
    """
    
    def __init__(self, name: str = "Custom Strategy"):
        self.name = name
        self.patterns_long = []
        self.patterns_short = []
        self.filters = []
        self.exit_rules = []
        
    def add_pattern_long(self, pattern_func, **kwargs):
        """Add a long pattern to the strategy"""
        self.patterns_long.append((pattern_func, kwargs))
        return self
    
    def add_pattern_short(self, pattern_func, **kwargs):
        """Add a short pattern to the strategy"""
        self.patterns_short.append((pattern_func, kwargs))
        return self
    
    def add_filter(self, filter_func, **kwargs):
        """Add a filter to the strategy"""
        self.filters.append((filter_func, kwargs))
        return self
    
    def generate_signals(self, df, sentido='long'):
        """
        Generate trading signals based on patterns and filters
        
        Parameters:
        -----------
        df : DataFrame
            Market data with OHLCV
        sentido : str
            'long', 'short', or 'both'
        
        Returns:
        --------
        DataFrame with 'signal' column
        """
        df = df.copy()
        df['signal'] = ''
        
        # Generate long signals
        if sentido in ['long', 'both'] and self.patterns_long:
            long_signal = pd.Series(False, index=df.index)
            for pattern_func, kwargs in self.patterns_long:
                long_signal |= pattern_func(df, **kwargs)
            
            # Apply filters
            for filter_func, kwargs in self.filters:
                filter_result = filter_func(df, **kwargs)
                long_signal &= filter_result
            
            df.loc[long_signal, 'signal'] = 'P'
        
        # Generate short signals
        if sentido in ['short', 'both'] and self.patterns_short:
            short_signal = pd.Series(False, index=df.index)
            for pattern_func, kwargs in self.patterns_short:
                short_signal |= pattern_func(df, **kwargs)
            
            # Apply filters
            for filter_func, kwargs in self.filters:
                filter_result = filter_func(df, **kwargs)
                short_signal &= filter_result
            
            df.loc[short_signal, 'signal'] = 'cP'
        
        # Shift to avoid lookahead bias
        df['signal'] = df['signal'].shift(1)
        df['signal'] = df['signal'].fillna('')
        
        return df

# Common filters
def filter_ma_trend(df, ma_fast=50, ma_slow=200, trend='bullish'):
    """Moving average trend filter"""
    if f'sma{ma_fast}' not in df.columns:
        df = ocpSma(df, ma_fast)
    if f'sma{ma_slow}' not in df.columns:
        df = ocpSma(df, ma_slow)
    
    if trend == 'bullish':
        return df[f'sma{ma_fast}'] > df[f'sma{ma_slow}']
    elif trend == 'bearish':
        return df[f'sma{ma_fast}'] < df[f'sma{ma_slow}']
    else:
        return pd.Series(True, index=df.index)

def filter_rsi(df, rsi_period=14, rsi_min=30, rsi_max=70):
    """RSI filter"""
    if f'rsi{rsi_period}' not in df.columns:
        df = ocpRsi(df, rsi_period)
    
    return (df[f'rsi{rsi_period}'] >= rsi_min) & (df[f'rsi{rsi_period}'] <= rsi_max)

print("✓ Strategy builder system defined")

---
## 5. Backtesting Engine

Core backtesting functions from the validation framework

In [ ]:
def damePosition(df):
    """Convert signals to position tracking"""
    df = df.copy()
    df['In'] = 0
    df['p'] = 0
    df['Out'] = 0
    
    in_position = False
    
    for i in range(len(df)):
        if df['signal'].iloc[i] in ['P', 'cP'] and not in_position:
            df.iloc[i, df.columns.get_loc('In')] = 1
            in_position = True
        elif in_position:
            df.iloc[i, df.columns.get_loc('p')] = 1
            if df['signal'].iloc[i] in ['cP', 'P']:
                df.iloc[i, df.columns.get_loc('Out')] = 1
                in_position = False
    
    return df

def dameSalidaVelas(df, num=0):
    """Exit after N bars"""
    if num == 0:
        return df
    
    df = df.copy()
    count = 0
    
    for i in range(len(df)):
        if df['In'].iloc[i] == 1:
            count = 1
        elif df['p'].iloc[i] == 1:
            count += 1
            if count >= num:
                df.iloc[i, df.columns.get_loc('Out')] = 1
                df.iloc[i, df.columns.get_loc('p')] = 0
                count = 0
        else:
            count = 0
    
    return df

def calcularRoi(capIn, capFn):
    """Calculate ROI"""
    if capIn == 0:
        return 0
    return ((capFn - capIn) / capIn) * 100

def pnlSalida(precioEntrada, precioSalida, comision, slippage, sentido):
    """Calculate P&L for a trade"""
    if sentido == 'long':
        pnl = (precioSalida - precioEntrada) - comision - slippage
    else:
        pnl = (precioEntrada - precioSalida) - comision - slippage
    return pnl

def dameSalidaPnl(df, sentido='long', tp=0, sl=0, comision=0, slippage=0):
    """Exit by P&L targets with commission and slippage"""
    df = df.copy()
    df['pnl'] = 0.0
    
    entry_price = 0
    in_trade = False
    
    for i in range(len(df)):
        if df['In'].iloc[i] == 1:
            entry_price = df['Close'].iloc[i]
            in_trade = True
        
        elif in_trade:
            hit_tp = False
            hit_sl = False
            
            if tp > 0:
                if sentido == 'long':
                    target_price = entry_price * (1 + tp/100)
                    if df['High'].iloc[i] >= target_price:
                        hit_tp = True
                        exit_price = target_price
                else:
                    target_price = entry_price * (1 - tp/100)
                    if df['Low'].iloc[i] <= target_price:
                        hit_tp = True
                        exit_price = target_price
            
            if sl > 0 and not hit_tp:
                if sentido == 'long':
                    stop_price = entry_price * (1 - sl/100)
                    if df['Low'].iloc[i] <= stop_price:
                        hit_sl = True
                        exit_price = stop_price
                else:
                    stop_price = entry_price * (1 + sl/100)
                    if df['High'].iloc[i] >= stop_price:
                        hit_sl = True
                        exit_price = stop_price
            
            if hit_tp or hit_sl:
                pnl = pnlSalida(entry_price, exit_price, comision, slippage, sentido)
                df.iloc[i, df.columns.get_loc('pnl')] = pnl
                df.iloc[i, df.columns.get_loc('Out')] = 1
                df.iloc[i, df.columns.get_loc('p')] = 0
                in_trade = False
            elif df['Out'].iloc[i] == 1:
                exit_price = df['Close'].iloc[i]
                pnl = pnlSalida(entry_price, exit_price, comision, slippage, sentido)
                df.iloc[i, df.columns.get_loc('pnl')] = pnl
                in_trade = False
    
    return df

def calculaCurvas(df, size=1):
    """Calculate equity curves"""
    df = df.copy()
    df['curvActivo'] = 100 * (df['Close'] / df['Close'].iloc[0])
    
    capital = 100
    curve = []
    
    for i in range(len(df)):
        if df['pnl'].iloc[i] != 0:
            pnl_pct = (df['pnl'].iloc[i] / df['Close'].iloc[i]) * 100
            capital = capital * (1 + (pnl_pct * size) / 100)
        curve.append(capital)
    
    df['curvSistema'] = curve
    return df

print("✓ Backtesting engine defined")

---
## 6. Performance Metrics

In [ ]:
def crearDfBacktesting():
    """Create empty backtesting results dataframe"""
    columns = ['nombre', 'Y', 'op', 'pos', 'neg', 'op/Y', 'mDIT', 'tInv%', 
               'pa%', 'capIn', 'capFn', 'roi%', 'cagr%', 'mPos%', 'mNeg%', 
               'em%', 'exca%', 'PF', 'Payf', 'shs', 'maxDD%', 'medDD%', 'OCP']
    return pd.DataFrame(columns=columns)

def backActivoList(df):
    """Calculate metrics for buy-and-hold"""
    years = len(df) / 252
    capIn = 100
    capFn = df['curvActivo'].iloc[-1]
    roi = calcularRoi(capIn, capFn)
    cagr = ((capFn / capIn) ** (1 / years) - 1) * 100 if years > 0 else 0
    
    # Calculate drawdown
    cummax = df['curvActivo'].cummax()
    drawdown = ((df['curvActivo'] - cummax) / cummax) * 100
    maxDD = drawdown.min()
    medDD = drawdown[drawdown < 0].mean() if len(drawdown[drawdown < 0]) > 0 else 0
    
    return ['Buy&Hold', years, 1, 0, 0, 0, len(df), 100, 0, 
            capIn, capFn, roi, cagr, 0, 0, 0, 0, 0, 0, 0, maxDD, medDD, 0]

def backSistemaList(df, nombre='Sistema'):
    """Calculate comprehensive system metrics"""
    years = len(df) / 252
    
    # Trade statistics
    trades = df[df['Out'] == 1].copy()
    num_trades = len(trades)
    
    if num_trades == 0:
        return [nombre] + [0] * 22
    
    winning_trades = trades[trades['pnl'] > 0]
    losing_trades = trades[trades['pnl'] < 0]
    
    pos = len(winning_trades)
    neg = len(losing_trades)
    op_per_year = num_trades / years if years > 0 else 0
    
    # Days in trade
    days_in_trade = []
    count = 0
    for i in range(len(df)):
        if df['In'].iloc[i] == 1:
            count = 1
        elif df['p'].iloc[i] == 1:
            count += 1
        elif df['Out'].iloc[i] == 1:
            count += 1
            days_in_trade.append(count)
            count = 0
    
    mDIT = np.mean(days_in_trade) if days_in_trade else 0
    tInv = (sum(days_in_trade) / len(df)) * 100 if len(df) > 0 else 0
    
    # Win rate
    win_rate = (pos / num_trades) * 100 if num_trades > 0 else 0
    
    # Capital metrics
    capIn = 100
    capFn = df['curvSistema'].iloc[-1]
    roi = calcularRoi(capIn, capFn)
    cagr = ((capFn / capIn) ** (1 / years) - 1) * 100 if years > 0 else 0
    
    # Average win/loss
    if pos > 0:
        avg_win = (winning_trades['pnl'].sum() / winning_trades['Close'].sum()) * 100
    else:
        avg_win = 0
    
    if neg > 0:
        avg_loss = (losing_trades['pnl'].sum() / losing_trades['Close'].sum()) * 100
    else:
        avg_loss = 0
    
    # Mathematical expectancy
    em = (win_rate/100 * avg_win) + ((1 - win_rate/100) * avg_loss)
    
    # Excursion (simplified)
    exca = avg_loss * 1.5 if avg_loss < 0 else 0
    
    # Profit Factor
    gross_profit = winning_trades['pnl'].sum() if pos > 0 else 0
    gross_loss = abs(losing_trades['pnl'].sum()) if neg > 0 else 0
    PF = gross_profit / gross_loss if gross_loss > 0 else 0
    
    # Payoff ratio
    Payf = abs(avg_win / avg_loss) if avg_loss != 0 else 0
    
    # Sharpe-like ratio
    returns = trades['pnl'] / trades['Close'] * 100
    shs = (returns.mean() / returns.std()) * np.sqrt(252) if len(returns) > 0 and returns.std() > 0 else 0
    
    # Drawdown
    cummax = df['curvSistema'].cummax()
    drawdown = ((df['curvSistema'] - cummax) / cummax) * 100
    maxDD = drawdown.min()
    medDD = drawdown[drawdown < 0].mean() if len(drawdown[drawdown < 0]) > 0 else 0
    
    # OCP (Optimal Composite Performance)
    OCP = cagr / abs(medDD) if medDD != 0 else 0
    
    return [nombre, years, num_trades, pos, neg, op_per_year, mDIT, tInv, 
            win_rate, capIn, capFn, roi, cagr, avg_win, avg_loss, em, exca, 
            PF, Payf, shs, maxDD, medDD, OCP]

def backAddList(dfBack, lista):
    """Add results to backtesting dataframe"""
    new_row = pd.DataFrame([lista], columns=dfBack.columns)
    return pd.concat([dfBack, new_row], ignore_index=True)

print("✓ Performance metrics defined")

---
## 7. Visualization Tools

In [ ]:
def dameGraficoBacktest(df, nombre='Sistema', velas=0, size=1):
    """Plot backtesting results"""
    if velas > 0:
        df = df.tail(velas)
    
    fig, axes = plt.subplots(4, 1, figsize=(15, 12))
    
    # Price and signals
    axes[0].plot(df.index, df['Close'], label='Price', linewidth=1)
    axes[0].scatter(df[df['In'] == 1].index, df[df['In'] == 1]['Close'], 
                    color='green', marker='^', s=100, label='Entry', zorder=5)
    axes[0].scatter(df[df['Out'] == 1].index, df[df['Out'] == 1]['Close'], 
                    color='red', marker='v', s=100, label='Exit', zorder=5)
    axes[0].set_title(f'{nombre} - Price and Signals', fontsize=14, fontweight='bold')
    axes[0].legend()
    axes[0].grid(True, alpha=0.3)
    
    # Equity curves
    axes[1].plot(df.index, df['curvActivo'], label='Buy & Hold', linewidth=2, alpha=0.7)
    axes[1].plot(df.index, df['curvSistema'], label='Strategy', linewidth=2)
    axes[1].set_title('Equity Curves', fontsize=14, fontweight='bold')
    axes[1].legend()
    axes[1].grid(True, alpha=0.3)
    axes[1].set_ylabel('Capital (%)')
    
    # Drawdown
    cummax = df['curvSistema'].cummax()
    drawdown = ((df['curvSistema'] - cummax) / cummax) * 100
    axes[2].fill_between(df.index, drawdown, 0, color='red', alpha=0.3)
    axes[2].plot(df.index, drawdown, color='darkred', linewidth=1)
    axes[2].set_title('Strategy Drawdown', fontsize=14, fontweight='bold')
    axes[2].set_ylabel('Drawdown (%)')
    axes[2].grid(True, alpha=0.3)
    
    # Trade P&L
    trades = df[df['Out'] == 1].copy()
    if len(trades) > 0:
        colors = ['green' if pnl > 0 else 'red' for pnl in trades['pnl']]
        axes[3].bar(range(len(trades)), trades['pnl'].values, color=colors, alpha=0.6)
        axes[3].axhline(y=0, color='black', linestyle='-', linewidth=0.5)
        axes[3].set_title('Trade P&L', fontsize=14, fontweight='bold')
        axes[3].set_ylabel('P&L')
        axes[3].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

def plot_strategy_comparison(results_df):
    """Compare multiple strategies"""
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))
    
    # Filter out Buy&Hold
    strategies = results_df[results_df['nombre'] != 'Buy&Hold'].copy()
    
    if len(strategies) == 0:
        print("No strategies to compare")
        return
    
    # CAGR comparison
    axes[0, 0].barh(strategies['nombre'], strategies['cagr%'])
    axes[0, 0].set_xlabel('CAGR (%)')
    axes[0, 0].set_title('Compound Annual Growth Rate', fontweight='bold')
    axes[0, 0].grid(True, alpha=0.3)
    
    # Win Rate
    axes[0, 1].barh(strategies['nombre'], strategies['pa%'])
    axes[0, 1].set_xlabel('Win Rate (%)')
    axes[0, 1].set_title('Win Rate', fontweight='bold')
    axes[0, 1].grid(True, alpha=0.3)
    
    # Profit Factor
    axes[1, 0].barh(strategies['nombre'], strategies['PF'])
    axes[1, 0].set_xlabel('Profit Factor')
    axes[1, 0].set_title('Profit Factor', fontweight='bold')
    axes[1, 0].grid(True, alpha=0.3)
    
    # Max Drawdown
    axes[1, 1].barh(strategies['nombre'], strategies['maxDD%'])
    axes[1, 1].set_xlabel('Max Drawdown (%)')
    axes[1, 1].set_title('Maximum Drawdown', fontweight='bold')
    axes[1, 1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

print("✓ Visualization tools defined")

---
## 8. Strategy Factory

Pre-built strategies for quick testing

In [ ]:
class StrategyFactory:
    """Factory for creating pre-configured strategies"""
    
    @staticmethod
    def create_sacudida_strategy(name="Sacudida", with_filter=False):
        """Create Sacudida pattern strategy"""
        strategy = StrategyBuilder(name)
        strategy.add_pattern_long(PatternLibrary.sacudida_long)
        strategy.add_pattern_short(PatternLibrary.sacudida_short)
        
        if with_filter:
            strategy.add_filter(filter_ma_trend, ma_fast=50, ma_slow=200, trend='bullish')
        
        return strategy
    
    @staticmethod
    def create_envolvente_strategy(name="Envolvente", with_filter=False):
        """Create Engulfing pattern strategy"""
        strategy = StrategyBuilder(name)
        strategy.add_pattern_long(PatternLibrary.envolvente_long)
        strategy.add_pattern_short(PatternLibrary.envolvente_short)
        
        if with_filter:
            strategy.add_filter(filter_ma_trend, ma_fast=50, ma_slow=200, trend='bullish')
        
        return strategy
    
    @staticmethod
    def create_volumen_strategy(name="Volumen Climático", vol_mult=1.75, with_filter=False):
        """Create Climatic Volume strategy"""
        strategy = StrategyBuilder(name)
        strategy.add_pattern_long(PatternLibrary.volumen_climatico_long, vol_multiplier=vol_mult)
        strategy.add_pattern_short(PatternLibrary.volumen_climatico_short, vol_multiplier=vol_mult)
        
        if with_filter:
            strategy.add_filter(filter_ma_trend, ma_fast=50, ma_slow=200, trend='bullish')
        
        return strategy
    
    @staticmethod
    def create_combined_strategy(name="Combined", patterns=['sacudida', 'envolvente'], with_filter=False):
        """Create strategy combining multiple patterns"""
        strategy = StrategyBuilder(name)
        
        for pattern in patterns:
            if pattern == 'sacudida':
                strategy.add_pattern_long(PatternLibrary.sacudida_long)
                strategy.add_pattern_short(PatternLibrary.sacudida_short)
            elif pattern == 'envolvente':
                strategy.add_pattern_long(PatternLibrary.envolvente_long)
                strategy.add_pattern_short(PatternLibrary.envolvente_short)
            elif pattern == 'volumen':
                strategy.add_pattern_long(PatternLibrary.volumen_climatico_long)
                strategy.add_pattern_short(PatternLibrary.volumen_climatico_short)
        
        if with_filter:
            strategy.add_filter(filter_ma_trend, ma_fast=50, ma_slow=200, trend='bullish')
        
        return strategy

print("✓ Strategy factory defined")

---
## 9. Multi-Strategy Backtesting System

In [ ]:
def run_strategy_backtest(df, strategy, sentido='long', tp=0, sl=0, 
                          comision=0, slippage=0, n_velas=0, size=1):
    """
    Run complete backtest for a strategy
    
    Parameters:
    -----------
    df : DataFrame
        Market data
    strategy : StrategyBuilder
        Strategy to test
    sentido : str
        'long', 'short', or 'both'
    tp : float
        Take profit %
    sl : float
        Stop loss %
    comision : float
        Commission per trade
    slippage : float
        Slippage per trade
    n_velas : int
        Number of bars for time-based exit (0 = disabled)
    size : float
        Position size multiplier
    
    Returns:
    --------
    tuple: (result_df, metrics_list)
    """
    # Generate signals
    df_strategy = strategy.generate_signals(df, sentido=sentido)
    
    # Convert to positions
    df_strategy = damePosition(df_strategy)
    
    # Apply exit rules
    if n_velas > 0:
        df_strategy = dameSalidaVelas(df_strategy, n_velas)
    
    df_strategy = dameSalidaPnl(df_strategy, sentido, tp, sl, comision, slippage)
    
    # Calculate curves
    df_strategy = calculaCurvas(df_strategy, size)
    
    # Calculate metrics
    metrics = backSistemaList(df_strategy, nombre=strategy.name)
    
    return df_strategy, metrics

def compare_strategies(df, strategies, sentido='long', tp=0, sl=0, 
                      comision=0, slippage=0, n_velas=0, size=1):
    """
    Compare multiple strategies on the same data
    
    Parameters:
    -----------
    df : DataFrame
        Market data
    strategies : list
        List of StrategyBuilder instances
    ... (same as run_strategy_backtest)
    
    Returns:
    --------
    DataFrame with comparison metrics
    """
    results = crearDfBacktesting()
    
    # Add buy & hold
    df_bh = df.copy()
    df_bh['curvActivo'] = 100 * (df_bh['Close'] / df_bh['Close'].iloc[0])
    df_bh['curvSistema'] = df_bh['curvActivo']
    bh_metrics = backActivoList(df_bh)
    results = backAddList(results, bh_metrics)
    
    # Test each strategy
    for strategy in strategies:
        print(f"Testing {strategy.name}...")
        df_result, metrics = run_strategy_backtest(
            df, strategy, sentido, tp, sl, comision, slippage, n_velas, size
        )
        results = backAddList(results, metrics)
    
    return results

print("✓ Multi-strategy backtesting system defined")

---
## 10. Multi-Asset & Multi-Parameter Optimization

In [ ]:
def optimize_strategy_parameters(df, base_strategy, param_grid, 
                                 sentido='long', tp=0, sl=0, 
                                 comision=0, slippage=0, n_velas=0, size=1,
                                 metric='OCP'):
    """
    Optimize strategy parameters using grid search
    
    Parameters:
    -----------
    df : DataFrame
        Market data
    base_strategy : str
        'sacudida', 'envolvente', 'volumen', or 'combined'
    param_grid : dict
        Parameters to optimize (e.g., {'tp': [1, 2, 3], 'sl': [1, 2, 3]})
    metric : str
        Metric to optimize ('OCP', 'cagr%', 'PF', 'shs')
    
    Returns:
    --------
    DataFrame with all parameter combinations and results
    """
    from itertools import product
    
    results = []
    
    # Generate all parameter combinations
    param_names = list(param_grid.keys())
    param_values = list(param_grid.values())
    
    total_combinations = 1
    for values in param_values:
        total_combinations *= len(values)
    
    print(f"Testing {total_combinations} parameter combinations...")
    
    for i, combination in enumerate(product(*param_values)):
        params = dict(zip(param_names, combination))
        
        # Create strategy
        if base_strategy == 'sacudida':
            strategy = StrategyFactory.create_sacudida_strategy(
                name=f"Sacudida_{i}",
                with_filter=params.get('with_filter', False)
            )
        elif base_strategy == 'envolvente':
            strategy = StrategyFactory.create_envolvente_strategy(
                name=f"Envolvente_{i}",
                with_filter=params.get('with_filter', False)
            )
        elif base_strategy == 'volumen':
            strategy = StrategyFactory.create_volumen_strategy(
                name=f"Volumen_{i}",
                vol_mult=params.get('vol_mult', 1.75),
                with_filter=params.get('with_filter', False)
            )
        else:
            strategy = StrategyFactory.create_combined_strategy(
                name=f"Combined_{i}",
                patterns=params.get('patterns', ['sacudida', 'envolvente']),
                with_filter=params.get('with_filter', False)
            )
        
        # Run backtest
        _, metrics = run_strategy_backtest(
            df, strategy, sentido,
            params.get('tp', tp),
            params.get('sl', sl),
            comision, slippage,
            params.get('n_velas', n_velas),
            size
        )
        
        # Add parameters to results
        result = {**params, **dict(zip(crearDfBacktesting().columns, metrics))}
        results.append(result)
        
        if (i + 1) % 10 == 0:
            print(f"Progress: {i + 1}/{total_combinations}")
    
    results_df = pd.DataFrame(results)
    
    # Sort by metric
    if metric in results_df.columns:
        results_df = results_df.sort_values(by=metric, ascending=False)
    
    print(f"\n✓ Optimization complete! Best {metric}: {results_df[metric].iloc[0]:.2f}")
    
    return results_df

def test_multiple_assets(symbols, strategies, start_date, end_date,
                        sentido='long', tp=0, sl=0, comision=0, slippage=0, 
                        n_velas=0, size=1):
    """
    Test strategies across multiple assets
    
    Parameters:
    -----------
    symbols : list
        List of ticker symbols
    strategies : list
        List of StrategyBuilder instances
    start_date : str
        Start date 'YYYY-MM-DD'
    end_date : str
        End date 'YYYY-MM-DD'
    
    Returns:
    --------
    Dict with results for each symbol
    """
    all_results = {}
    
    for symbol in symbols:
        print(f"\n{'='*60}")
        print(f"Testing {symbol}")
        print(f"{'='*60}")
        
        # Download data
        df = yf.download(symbol, start=start_date, end=end_date, progress=False)
        
        if len(df) == 0:
            print(f"No data available for {symbol}")
            continue
        
        # Test strategies
        results = compare_strategies(df, strategies, sentido, tp, sl, 
                                    comision, slippage, n_velas, size)
        
        all_results[symbol] = results
        
        print(f"\n{symbol} Results:")
        print(results[['nombre', 'op', 'pa%', 'cagr%', 'PF', 'maxDD%', 'OCP']].to_string(index=False))
    
    return all_results

print("✓ Optimization framework defined")

---
---
# EXAMPLES AND USAGE
---
---

## Example 1: Load Data and Test Single Strategy

In [ ]:
# Download sample data
symbol = 'AAPL'
start_date = '2020-01-01'
end_date = '2023-12-31'

print(f"Downloading {symbol} data...")
df = yf.download(symbol, start=start_date, end=end_date)
print(f"✓ Downloaded {len(df)} bars")

# Add required indicators
df = ocpSma(df, 50)
df = ocpSma(df, 200)
df = ocpVolumeSma(df, 20)

print(f"\nData shape: {df.shape}")
print(f"Date range: {df.index[0]} to {df.index[-1]}")

In [ ]:
# Create and test Sacudida strategy
sacudida = StrategyFactory.create_sacudida_strategy(
    name="Sacudida Long",
    with_filter=True  # Use MA trend filter
)

# Run backtest
df_result, metrics = run_strategy_backtest(
    df, 
    sacudida, 
    sentido='long',
    tp=2,           # 2% take profit
    sl=1,           # 1% stop loss
    comision=0.1,   # 0.1% commission
    slippage=0.05,  # 0.05% slippage
    n_velas=5,      # Exit after 5 bars
    size=1
)

# Display results
results_df = crearDfBacktesting()
results_df = backAddList(results_df, metrics)
print("\nStrategy Performance:")
print(results_df.T)

In [ ]:
# Visualize results
dameGraficoBacktest(df_result, nombre="Sacudida Long", velas=252)  # Last year

## Example 2: Compare Multiple Strategies

In [ ]:
# Create multiple strategies
strategies = [
    StrategyFactory.create_sacudida_strategy("Sacudida", with_filter=False),
    StrategyFactory.create_sacudida_strategy("Sacudida + Filter", with_filter=True),
    StrategyFactory.create_envolvente_strategy("Envolvente", with_filter=False),
    StrategyFactory.create_envolvente_strategy("Envolvente + Filter", with_filter=True),
    StrategyFactory.create_volumen_strategy("Volumen 1.75x", vol_mult=1.75, with_filter=False),
    StrategyFactory.create_volumen_strategy("Volumen 2.0x", vol_mult=2.0, with_filter=False),
    StrategyFactory.create_combined_strategy("All Patterns", 
                                            patterns=['sacudida', 'envolvente', 'volumen'],
                                            with_filter=True)
]

# Compare all strategies
comparison = compare_strategies(
    df, 
    strategies,
    sentido='long',
    tp=2,
    sl=1,
    comision=0.1,
    slippage=0.05,
    n_velas=0,  # No time-based exit
    size=1
)

# Display comparison
print("\nStrategy Comparison:")
print(comparison[['nombre', 'op', 'pa%', 'cagr%', 'PF', 'maxDD%', 'OCP']].to_string(index=False))

In [ ]:
# Visualize comparison
plot_strategy_comparison(comparison)

## Example 3: Parameter Optimization

In [ ]:
# Define parameter grid
param_grid = {
    'tp': [1, 2, 3, 4, 5],
    'sl': [0.5, 1, 1.5, 2],
    'with_filter': [False, True]
}

# Optimize Sacudida strategy
optimization_results = optimize_strategy_parameters(
    df,
    base_strategy='sacudida',
    param_grid=param_grid,
    sentido='long',
    comision=0.1,
    slippage=0.05,
    metric='OCP'  # Optimize for OCP
)

# Display top 10 parameter combinations
print("\nTop 10 Parameter Combinations:")
print(optimization_results[['tp', 'sl', 'with_filter', 'op', 'pa%', 'cagr%', 'PF', 'maxDD%', 'OCP']].head(10).to_string(index=False))

In [ ]:
# Visualize optimization results (heatmap)
pivot_table = optimization_results.pivot_table(
    values='OCP',
    index='tp',
    columns='sl',
    aggfunc='mean'
)

plt.figure(figsize=(10, 8))
sns.heatmap(pivot_table, annot=True, fmt='.2f', cmap='RdYlGn', center=0)
plt.title('OCP Optimization Heatmap (TP vs SL)', fontsize=14, fontweight='bold')
plt.xlabel('Stop Loss (%)')
plt.ylabel('Take Profit (%)')
plt.tight_layout()
plt.show()

## Example 4: Multi-Asset Testing

In [ ]:
# Define assets to test
symbols = ['AAPL', 'MSFT', 'GOOGL', 'TSLA', 'SPY']

# Define strategies to test
test_strategies = [
    StrategyFactory.create_sacudida_strategy("Sacudida", with_filter=True),
    StrategyFactory.create_envolvente_strategy("Envolvente", with_filter=True),
    StrategyFactory.create_combined_strategy("Combined", 
                                            patterns=['sacudida', 'envolvente'],
                                            with_filter=True)
]

# Test across all assets
multi_asset_results = test_multiple_assets(
    symbols,
    test_strategies,
    start_date='2020-01-01',
    end_date='2023-12-31',
    sentido='long',
    tp=2,
    sl=1,
    comision=0.1,
    slippage=0.05,
    size=1
)

In [ ]:
# Create summary comparison across assets
summary_data = []
for symbol, results in multi_asset_results.items():
    for _, row in results.iterrows():
        if row['nombre'] != 'Buy&Hold':
            summary_data.append({
                'Asset': symbol,
                'Strategy': row['nombre'],
                'CAGR%': row['cagr%'],
                'Win%': row['pa%'],
                'PF': row['PF'],
                'MaxDD%': row['maxDD%'],
                'OCP': row['OCP']
            })

summary_df = pd.DataFrame(summary_data)
print("\nMulti-Asset Summary:")
print(summary_df.to_string(index=False))

# Best strategy per asset
print("\n" + "="*60)
print("Best Strategy per Asset (by OCP):")
print("="*60)
best_per_asset = summary_df.loc[summary_df.groupby('Asset')['OCP'].idxmax()]
print(best_per_asset.to_string(index=False))

## Example 5: Custom Strategy Creation

In [ ]:
# Create a completely custom strategy
custom_strategy = StrategyBuilder(name="My Custom Strategy")

# Add Sacudida long pattern
custom_strategy.add_pattern_long(PatternLibrary.sacudida_long)

# Add Envolvente short pattern
custom_strategy.add_pattern_short(PatternLibrary.envolvente_short)

# Add MA trend filter (only bullish trend)
custom_strategy.add_filter(filter_ma_trend, ma_fast=50, ma_slow=200, trend='bullish')

# Test custom strategy
df_custom, metrics_custom = run_strategy_backtest(
    df,
    custom_strategy,
    sentido='both',  # Both long and short
    tp=3,
    sl=1.5,
    comision=0.1,
    slippage=0.05,
    n_velas=0,
    size=1
)

# Display results
custom_results = crearDfBacktesting()
custom_results = backAddList(custom_results, metrics_custom)
print("\nCustom Strategy Performance:")
print(custom_results[['nombre', 'op', 'pa%', 'cagr%', 'PF', 'maxDD%', 'OCP']].T)

---
## Summary and Next Steps

### What This Framework Provides:

1. **Pattern Library**: All three patterns from Algo Strategy Builder
   - Sacudida (Shake-out)
   - Envolvente (Engulfing)
   - Volumen Climático (Climatic Volume)

2. **Flexible Strategy Builder**: Mix and match patterns, filters, and parameters

3. **Comprehensive Backtesting**: Full performance metrics and visualization

4. **Multi-Strategy Comparison**: Test multiple strategies simultaneously

5. **Parameter Optimization**: Grid search for best parameters

6. **Multi-Asset Testing**: Validate across different instruments

### Recommended Workflow:

1. **Explore**: Test individual patterns on your data
2. **Compare**: Test all patterns to find what works best
3. **Optimize**: Find optimal parameters for your best patterns
4. **Validate**: Test on multiple assets and timeframes
5. **Refine**: Create custom combinations based on results

### Next Steps:

- Add validation methods (Monte Carlo, Walk-Forward, Cross-Validation)
- Implement more advanced filters (volatility, volume, etc.)
- Add more exit strategies (trailing stop, partial exits)
- Create live trading integration
- Add risk management (position sizing, portfolio allocation)

---